<a href="https://colab.research.google.com/github/leonardo69-code/Python---Banco-de-Dados/blob/main/Gabarito_de_Subqueries_AND_CTEs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mysql-connector-python

In [ ]:
import mysql.connector
import pandas as pd

In [ ]:
host = '34.70.26.188' # endereço do banco externo que eu quero acessar
user = 'root'
password = ''
database = 'locadora'

In [ ]:
!curl ipecho.net/plain

34.106.57.192

In [ ]:
def consulta():
  query = input('Informe o código da query: ')
  tabela = input('Informe o nome da tabela pandas para armazenar o resultado: ')
  connection = mysql.connector.connect(
      host = host,
      user = user,
      password = password,
      database = database
  )
  cursor = connection.cursor()

  try:
    cursor.execute(query)
    result = cursor.fetchall()
    globals()[tabela] = pd.DataFrame(result, columns=cursor.column_names)
    display(globals()[tabela])
  finally:
    cursor.close()
    connection.close()

In [ ]:
def ddl():
    comando = input('Informe o comando DDL: ')
    connection = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    cursor = connection.cursor()

    try:
        cursor.execute(comando)
        connection.commit()  # Confirma as mudanças no banco
        print(f"Comando DDL executado com sucesso:\n{comando}")
    except mysql.connector.Error as err:
        print(f"Erro ao executar o comando DDL: {err}")
    finally:
        cursor.close()
        connection.close()

In [ ]:
consulta()

Informe o código da query: show tables;
Informe o nome da tabela pandas para armazenar o resultado: 


,Tables_in_locadora
0,aluguel
1,carro
2,carro_aluguel
3,cliente
4,marca
5,modelo_valor


In [ ]:
'''
1 - Lista de cidades com clientes que já alugaram carros
Liste todas as cidades em que há clientes que realizaram aluguéis.
Dica: Use uma subconsulta para verificar clientes na tabela aluguel.

Resposta:
SELECT DISTINCT c.cidade
FROM cliente c
WHERE c.codcliente IN (
    SELECT a.codcliente
    FROM aluguel a
);
'''
consulta()

Informe o código da query: SELECT DISTINCT c.cidade FROM cliente c WHERE c.codcliente IN (     SELECT a.codcliente     FROM aluguel a );
Informe o nome da tabela pandas para armazenar o resultado: 


,cidade
0,Duque de Caxias
1,Niterói
2,São Paulo


In [ ]:
'''
2 - Modelos de carros mais caros que a média:
Liste os modelos de carros cujo valor seja maior que o valor médio de todos
os carros.

SELECT modelo
FROM carro
WHERE valor > (SELECT AVG(valor) FROM carro);
'''
consulta()

Informe o código da query: SELECT modelo FROM carro WHERE valor > (SELECT AVG(valor) FROM carro);
Informe o nome da tabela pandas para armazenar o resultado: 


,modelo
0,Argo
1,Onix
2,Polo


In [ ]:
'''
3 - Clientes que alugaram carros da marca Ford
Liste os nomes dos clientes que alugaram pelo menos um carro da marca Ford.
Dica: Use uma subconsulta para filtrar a marca.

SELECT DISTINCT c.nome
FROM cliente c
JOIN aluguel a ON c.codcliente = a.codcliente
JOIN carro ca ON a.codcarro = ca.codcarro
WHERE ca.codmarca = (SELECT codmarca FROM marca WHERE marca = 'Ford');
'''
consulta()

Informe o código da query: SELECT DISTINCT c.nome FROM cliente c JOIN aluguel a ON c.codcliente = a.codcliente JOIN carro ca ON a.codcarro = ca.codcarro WHERE ca.codmarca = (SELECT codmarca FROM marca WHERE marca = 'Ford');
Informe o nome da tabela pandas para armazenar o resultado: 


,nome
0,Bruna Pereira
1,Ana Silva
2,Túlio Nascimento


In [ ]:
'''
4 - Carros alugados apenas por clientes de São Paulo
Liste os modelos de carros que foram alugados exclusivamente por clientes do
estado de São Paulo.
Dica: Use NOT IN com uma subconsulta.
SELECT DISTINCT ca.modelo
FROM carro ca
JOIN aluguel a ON ca.codcarro = a.codcarro
JOIN cliente c ON a.codcliente = c.codcliente
WHERE c.estado = 'SP'
  AND ca.codcarro NOT IN (
      SELECT DISTINCT a.codcarro
      FROM aluguel a
      JOIN cliente c ON a.codcliente = c.codcliente
      WHERE c.estado != 'SP'
  );
'''
consulta()

Informe o código da query: SELECT DISTINCT ca.modelo FROM carro ca JOIN aluguel a ON ca.codcarro = a.codcarro JOIN cliente c ON a.codcliente = c.codcliente WHERE c.estado = 'SP'   AND ca.codcarro NOT IN (       SELECT DISTINCT a.codcarro       FROM aluguel a       JOIN cliente c ON a.codcliente = c.codcliente       WHERE c.estado != 'SP'   );
Informe o nome da tabela pandas para armazenar o resultado: 


,modelo


In [ ]:
'''
5 - Datas mais movimentadas
Liste as datas em que houve mais de 2 aluguéis.
Dica: Use uma subconsulta com GROUP BY e HAVING.

SELECT data_aluguel
FROM aluguel
GROUP BY data_aluguel
HAVING COUNT(*) > 2;
'''
consulta()

Informe o código da query: SELECT data_aluguel FROM aluguel GROUP BY data_aluguel HAVING COUNT(*) > 2;
Informe o nome da tabela pandas para armazenar o resultado: 


,data_aluguel


In [ ]:
'''
6 - Clientes que nunca alugaram carros da marca Fiat
Liste os nomes dos clientes que nunca alugaram carros da marca Fiat.
SELECT nome
FROM cliente
WHERE codcliente NOT IN (
    SELECT DISTINCT a.codcliente
    FROM aluguel a
    JOIN carro ca ON a.codcarro = ca.codcarro
    WHERE ca.codmarca = (SELECT codmarca FROM marca WHERE marca = 'Fiat')
);
'''
consulta()

Informe o código da query: SELECT nome FROM cliente WHERE codcliente NOT IN (     SELECT DISTINCT a.codcliente     FROM aluguel a     JOIN carro ca ON a.codcarro = ca.codcarro     WHERE ca.codmarca = (SELECT codmarca FROM marca WHERE marca = 'Fiat') );
Informe o nome da tabela pandas para armazenar o resultado: 


,nome
0,Ana Silva
1,Bruna Pereira
2,Fernando Souza


In [ ]:
'''
7 - Carros mais alugados em cidades específicas
Liste os modelos de carros que foram alugados em cidades onde residem mais de
3 clientes.
Dica: Use uma subconsulta para encontrar essas cidades.

SELECT ca.modelo
FROM carro ca
JOIN aluguel a ON ca.codcarro = a.codcarro
JOIN cliente c ON a.codcliente = c.codcliente
WHERE c.cidade IN (
    SELECT cidade
    FROM cliente
    GROUP BY cidade
    HAVING COUNT(*) > 3
);
'''
consulta()

Informe o código da query: SELECT ca.modelo FROM carro ca JOIN aluguel a ON ca.codcarro = a.codcarro JOIN cliente c ON a.codcliente = c.codcliente WHERE c.cidade IN (     SELECT cidade     FROM cliente     GROUP BY cidade     HAVING COUNT(*) > 3 );
Informe o nome da tabela pandas para armazenar o resultado: 


,modelo


In [ ]:
'''
8 - Valor total de aluguéis por cliente com subconsulta
Liste os nomes dos clientes e o valor total gasto em aluguéis.
Dica: Use uma subconsulta para calcular a soma por cliente.

SELECT c.nome,
       (SELECT SUM(ca.valor)
        FROM aluguel a
        JOIN carro ca ON a.codcarro = ca.codcarro
        WHERE a.codcliente = c.codcliente) AS total_gasto
FROM cliente c;
'''
consulta()

Informe o código da query: SELECT c.nome,         (SELECT SUM(ca.valor)          FROM aluguel a          JOIN carro ca ON a.codcarro = ca.codcarro         WHERE a.codcliente = c.codcliente) AS total_gasto FROM cliente c;
Informe o nome da tabela pandas para armazenar o resultado: 


,nome,total_gasto
0,Ana Silva,400.0
1,Bruna Pereira,370.0
2,Túlio Nascimento,250.0
3,Fernando Souza,NaN
4,Lúcia Andrade,300.0


In [ ]:
'''
9 - Clientes que gastaram acima da média Liste os nomes dos clientes cujo valor
total gasto com aluguéis está acima da média geral.

WITH total_gasto AS (
    SELECT c.codcliente, SUM(ca.valor) AS total
    FROM cliente c
    JOIN aluguel a ON c.codcliente = a.codcliente
    JOIN carro ca ON a.codcarro = ca.codcarro
    GROUP BY c.codcliente
)
SELECT c.nome
FROM cliente c
JOIN total_gasto tg ON c.codcliente = tg.codcliente
WHERE tg.total > (SELECT AVG(total) FROM total_gasto);
'''
consulta()

Informe o código da query: WITH total_gasto AS (     SELECT c.codcliente, SUM(ca.valor) AS total     FROM cliente c     JOIN aluguel a ON c.codcliente = a.codcliente     JOIN carro ca ON a.codcarro = ca.codcarro     GROUP BY c.codcliente ) SELECT c.nome FROM cliente c JOIN total_gasto tg ON c.codcliente = tg.codcliente WHERE tg.total > (SELECT AVG(total) FROM total_gasto);
Informe o nome da tabela pandas para armazenar o resultado: 


,nome
0,Ana Silva
1,Bruna Pereira


In [ ]:
'''
10 - Clientes que mais alugaram em um único dia
Liste os nomes dos clientes que
realizaram o maior número de aluguéis em um único dia.
Dica: Use MAX em uma subconsulta.

WITH alugueis_por_dia AS (
    SELECT a.codcliente, COUNT(*) AS total
    FROM aluguel a
    GROUP BY a.codcliente, a.data_aluguel
)
SELECT c.nome
FROM cliente c
JOIN alugueis_por_dia apd ON c.codcliente = apd.codcliente
WHERE apd.total = (SELECT MAX(total) FROM alugueis_por_dia);
'''
consulta()

Informe o código da query: WITH alugueis_por_dia AS (     SELECT a.codcliente, COUNT(*) AS total     FROM aluguel a     GROUP BY a.codcliente, a.data_aluguel ) SELECT c.nome FROM cliente c JOIN alugueis_por_dia apd ON c.codcliente = apd.codcliente WHERE apd.total = (SELECT MAX(total) FROM alugueis_por_dia);
Informe o nome da tabela pandas para armazenar o resultado: 


,nome
0,Túlio Nascimento
1,Bruna Pereira
2,Bruna Pereira
3,Bruna Pereira
4,Ana Silva
5,Ana Silva
6,Ana Silva
7,Lúcia Andrade
8,Lúcia Andrade
9,Túlio Nascimento


In [ ]:
'''
11 - Clientes que alugaram todos os carros de uma marca
Liste os nomes dos clientes que alugaram todos os carros disponíveis da marca
Chevrolet.
Dica: Compare a contagem de carros alugados por cliente com a contagem total
de carros da marca.

WITH carros_chevrolet AS (
    SELECT codcarro
    FROM carro
    WHERE codmarca = (SELECT codmarca FROM marca WHERE marca = 'Chevrolet')
)
SELECT c.nome
FROM cliente c
WHERE NOT EXISTS (
    SELECT codcarro
    FROM carros_chevrolet
    WHERE codcarro NOT IN (
        SELECT a.codcarro
        FROM aluguel a
        WHERE a.codcliente = c.codcliente
    )
);
'''
consulta()

Informe o código da query: WITH carros_chevrolet AS (     SELECT codcarro     FROM carro     WHERE codmarca = (SELECT codmarca FROM marca WHERE marca = 'Chevrolet') ) SELECT c.nome FROM cliente c WHERE NOT EXISTS (     SELECT codcarro     FROM carros_chevrolet     WHERE codcarro NOT IN (         SELECT a.codcarro         FROM aluguel a         WHERE a.codcliente = c.codcliente     ) );
Informe o nome da tabela pandas para armazenar o resultado: 


,nome
0,Bruna Pereira


In [ ]:
'''
12 - Carros alugados mais de uma vez no mesmo dia Liste os modelos de carros
que foram alugados mais de uma vez na mesma data.

SELECT ca.modelo, a.data_aluguel
FROM aluguel a
JOIN carro ca ON a.codcarro = ca.codcarro
GROUP BY ca.modelo, a.data_aluguel
HAVING COUNT(*) > 1;
'''
consulta()

Informe o código da query: SELECT ca.modelo, a.data_aluguel FROM aluguel a JOIN carro ca ON a.codcarro = ca.codcarro GROUP BY ca.modelo, a.data_aluguel HAVING COUNT(*) > 1;
Informe o nome da tabela pandas para armazenar o resultado: 


,modelo,data_aluguel


In [ ]:
'''
13 - Carros mais caros alugados por clientes de um estado
Liste os modelos dos carros mais caros alugados por clientes do estado do
Rio de Janeiro.

SELECT DISTINCT ca.modelo
FROM aluguel a
JOIN carro ca ON a.codcarro = ca.codcarro
JOIN cliente c ON a.codcliente = c.codcliente
WHERE c.estado = 'RJ' AND ca.valor = (SELECT MAX(valor) FROM carro);
'''
consulta()

Informe o código da query: SELECT DISTINCT ca.modelo FROM aluguel a JOIN carro ca ON a.codcarro = ca.codcarro JOIN cliente c ON a.codcliente = c.codcliente WHERE c.estado = 'RJ' AND ca.valor = (SELECT MAX(valor) FROM carro);
Informe o nome da tabela pandas para armazenar o resultado: 


,modelo
0,Onix


In [ ]:
'''
14 - Datas de aluguel de carros populares
Liste as datas em que foram alugados carros das marcas Ford ou Volkswagen.

SELECT DISTINCT a.data_aluguel
FROM aluguel a
JOIN carro ca ON a.codcarro = ca.codcarro
WHERE ca.codmarca IN (
    SELECT codmarca
    FROM marca
    WHERE marca IN ('Ford', 'Volkswagen')
);
'''
consulta()

Informe o código da query: SELECT DISTINCT a.data_aluguel FROM aluguel a JOIN carro ca ON a.codcarro = ca.codcarro WHERE ca.codmarca IN (     SELECT codmarca     FROM marca     WHERE marca IN ('Ford', 'Volkswagen') );
Informe o nome da tabela pandas para armazenar o resultado: 


,data_aluguel
0,2023-04-02
1,2023-04-03
2,2023-04-15
3,2023-04-25
4,2023-04-05
5,2023-04-13


In [ ]:
'''
15 - Carros nunca alugados
Liste os modelos de carros que nunca foram alugados.

SELECT modelo
FROM carro
WHERE codcarro NOT IN (
    SELECT DISTINCT codcarro
    FROM aluguel
);
'''
consulta()

Informe o código da query: SELECT modelo FROM carro WHERE codcarro NOT IN (     SELECT DISTINCT codcarro     FROM aluguel );
Informe o nome da tabela pandas para armazenar o resultado: 


,modelo
0,Kwid


In [ ]:
'''
16 - Total de aluguéis por cliente
Crie uma CTE para calcular o número total de aluguéis realizados por cada
cliente e liste os clientes que alugaram mais de 2 vezes.

WITH total_alugueis AS (
    SELECT codcliente, COUNT(*) AS total
    FROM aluguel
    GROUP BY codcliente
)
SELECT c.nome, t.total
FROM cliente c
JOIN total_alugueis t ON c.codcliente = t.codcliente
WHERE t.total > 2;
'''
consulta()

Informe o código da query: WITH total_alugueis AS (     SELECT codcliente, COUNT(*) AS total     FROM aluguel     GROUP BY codcliente ) SELECT c.nome, t.total FROM cliente c JOIN total_alugueis t ON c.codcliente = t.codcliente WHERE t.total > 2;
Informe o nome da tabela pandas para armazenar o resultado: 


,nome,total
0,Ana Silva,3
1,Bruna Pereira,3


In [ ]:
'''
17 - Total gasto por cliente Crie uma CTE para calcular o valor total gasto por
cliente e liste apenas os clientes que gastaram mais de R$300,00.

WITH total_gasto AS (
    SELECT c.codcliente, SUM(ca.valor) AS total
    FROM cliente c
    JOIN aluguel a ON c.codcliente = a.codcliente
    JOIN carro ca ON a.codcarro = ca.codcarro
    GROUP BY c.codcliente
)
SELECT c.nome, tg.total
FROM cliente c
JOIN total_gasto tg ON c.codcliente = tg.codcliente
WHERE tg.total > 300.00;
'''
consulta()

Informe o código da query: WITH total_gasto AS (     SELECT c.codcliente, SUM(ca.valor) AS total     FROM cliente c     JOIN aluguel a ON c.codcliente = a.codcliente     JOIN carro ca ON a.codcarro = ca.codcarro     GROUP BY c.codcliente ) SELECT c.nome, tg.total FROM cliente c JOIN total_gasto tg ON c.codcliente = tg.codcliente WHERE tg.total > 300.00;
Informe o nome da tabela pandas para armazenar o resultado: 


,nome,total
0,Ana Silva,400.0
1,Bruna Pereira,370.0


In [ ]:
'''
Carros mais alugados
Crie uma CTE para calcular o número de vezes que cada modelo de carro foi
alugado e liste apenas os modelos alugados mais de 3 vezes.

WITH alugueis_por_carro AS (
    SELECT ca.modelo, COUNT(*) AS total
    FROM carro ca
    JOIN aluguel a ON ca.codcarro = a.codcarro
    GROUP BY ca.modelo
)
SELECT modelo, total
FROM alugueis_por_carro
WHERE total > 3;
'''
consulta()

Informe o código da query: WITH alugueis_por_carro AS (     SELECT ca.modelo, COUNT(*) AS total     FROM carro ca     JOIN aluguel a ON ca.codcarro = a.codcarro     GROUP BY ca.modelo ) SELECT modelo, total FROM alugueis_por_carro WHERE total > 3;
Informe o nome da tabela pandas para armazenar o resultado: 


,modelo,total
0,Ka,4


In [ ]:
'''
19 - Estados com maior número de clientes
Crie uma CTE para calcular o número de clientes por estado e liste os estados
com mais de 3 clientes.

WITH clientes_por_estado AS (
    SELECT estado, COUNT(*) AS total
    FROM cliente
    GROUP BY estado
)
SELECT estado, total
FROM clientes_por_estado
WHERE total > 3;
'''
consulta()

Informe o código da query: WITH clientes_por_estado AS (     SELECT estado, COUNT(*) AS total     FROM cliente     GROUP BY estado ) SELECT estado, total FROM clientes_por_estado WHERE total > 3;
Informe o nome da tabela pandas para armazenar o resultado: 


,estado,total


In [ ]:
'''
20 - Clientes com aluguéis consecutivos
Crie uma CTE para listar os clientes que realizaram aluguéis em dias
consecutivos.
Dica: Use a função DATEDIFF.

WITH alugueis_ordenados AS (
    SELECT codcliente, data_aluguel,
           LAG(data_aluguel) OVER (PARTITION BY codcliente ORDER BY data_aluguel) AS data_anterior
    FROM aluguel
),
alugueis_consecutivos AS (
    SELECT codcliente
    FROM alugueis_ordenados
    WHERE DATEDIFF(data_aluguel, data_anterior) = 1
)
SELECT DISTINCT c.nome
FROM cliente c
JOIN alugueis_consecutivos ac ON c.codcliente = ac.codcliente;
'''
consulta()

Informe o código da query: WITH alugueis_ordenados AS (     SELECT codcliente, data_aluguel,            LAG(data_aluguel) OVER (PARTITION BY codcliente ORDER BY data_aluguel) AS data_anterior     FROM aluguel ), alugueis_consecutivos AS (     SELECT codcliente     FROM alugueis_ordenados     WHERE DATEDIFF(data_aluguel, data_anterior) = 1 ) SELECT DISTINCT c.nome FROM cliente c JOIN alugueis_consecutivos ac ON c.codcliente = ac.codcliente;
Informe o nome da tabela pandas para armazenar o resultado: 


,nome
0,Bruna Pereira
